In [27]:
import pandas as pd
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import numpy as np
import re

np.random.seed(90042)
# import nltk
# nltk.download('wordnet')

from gensim import corpora, models

result = pd.read_csv('tweet_covid_result_bert.csv')
covid = pd.read_csv('./project-data/tweet_covid_bert.tsv')

In [28]:
new_data = pd.DataFrame()
new_data['sentence'] = covid['sentence'].copy()
new_data['Predicted'] = result['Predicted'].copy()
new_data_rumor = new_data[new_data['Predicted'] == 1]
new_data_nonrumor = new_data[new_data['Predicted'] == 0]

In [31]:



stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


def get_LDA(new_data=new_data):
    processed_data = new_data['sentence'].map(preprocess)
    dictionary = gensim.corpora.Dictionary(processed_data)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_data]

    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]

    print('###################LDA BOW######################')
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=None,per_word_topics=True)
    for idx, topic in lda_model.print_topics(-1):
        print('Topic: {} \nWords: {}'.format(idx, topic))
        s = re.findall("[A-Za-z'-]+",topic)
        word_str = ' '.join([str(elem) for elem in s])
        print('Topic: {} \nWords: {}'.format(idx, word_str))
    print('###################LDA TFIDF######################')
    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=None,per_word_topics=True)
    for idx, topic in lda_model_tfidf.print_topics(-1):
        print('Topic: {} Word: {}'.format(idx, topic))
        s = re.findall("[A-Za-z'-]+",topic)
        word_str = ' '.join([str(elem) for elem in s])
        print('Topic: {} \nWords: {}'.format(idx, word_str))

In [32]:
get_LDA(new_data_rumor)

###################LDA BOW######################
Topic: 0 
Words: 0.026*"say" + 0.021*"peopl" + 0.020*"trump" + 0.017*"death" + 0.015*"coronavirus" + 0.012*"go" + 0.012*"right" + 0.012*"think" + 0.011*"like" + 0.011*"american"
Topic: 0 
Words: say peopl trump death coronavirus go right think like american
Topic: 1 
Words: 0.041*"peopl" + 0.024*"like" + 0.020*"coronavirus" + 0.016*"trump" + 0.015*"test" + 0.011*"think" + 0.010*"go" + 0.010*"know" + 0.009*"american" + 0.009*"virus"
Topic: 1 
Words: peopl like coronavirus trump test think go know american virus
Topic: 2 
Words: 0.061*"coronavirus" + 0.028*"peopl" + 0.022*"like" + 0.017*"go" + 0.013*"protest" + 0.012*"know" + 0.011*"pandem" + 0.009*"kill" + 0.009*"need" + 0.009*"week"
Topic: 2 
Words: coronavirus peopl like go protest know pandem kill need week
Topic: 3 
Words: 0.029*"coronavirus" + 0.024*"american" + 0.023*"trump" + 0.020*"peopl" + 0.016*"death" + 0.014*"case" + 0.014*"presid" + 0.013*"die" + 0.012*"kill" + 0.012*"play"
T

In [33]:
get_LDA(new_data_nonrumor)

###################LDA BOW######################
Topic: 0 
Words: 0.016*"test" + 0.015*"trump" + 0.011*"peopl" + 0.011*"death" + 0.009*"american" + 0.007*"pandem" + 0.007*"need" + 0.007*"case" + 0.007*"know" + 0.005*"say"
Topic: 0 
Words: test trump peopl death american pandem need case know say
Topic: 1 
Words: 0.015*"case" + 0.013*"peopl" + 0.013*"trump" + 0.010*"american" + 0.009*"death" + 0.008*"virus" + 0.008*"like" + 0.008*"china" + 0.008*"pandem" + 0.007*"world"
Topic: 1 
Words: case peopl trump american death virus like china pandem world
Topic: 2 
Words: 0.024*"death" + 0.020*"trump" + 0.014*"case" + 0.014*"peopl" + 0.012*"test" + 0.012*"state" + 0.007*"american" + 0.006*"presid" + 0.006*"say" + 0.006*"number"
Topic: 2 
Words: death trump case peopl test state american presid say number
Topic: 3 
Words: 0.017*"trump" + 0.017*"death" + 0.013*"peopl" + 0.011*"test" + 0.008*"case" + 0.007*"american" + 0.007*"time" + 0.006*"number" + 0.006*"know" + 0.006*"say"
Topic: 3 
Words: tru